# AMIA 2016 Annual Symposium Workshop (WG13)

## Automated and Scalable Cloud-based RNA-Seq Data Analysis, Part II


Riyue Bao, Ph.D. 
Center for Research Informatics,
The University of Chicago.
November 13, 2016

***

## Objective

* Explore the quality metrics and input/output of the RNAseq pipeline
* Visualize result files and quality plots
* Pracice differential gene identification and pathway analysis

***

## Dataset

The test datasets used in this workshop are from 
Fog. et al. 2015. Loss of PRDM11 promotes MYC-driven lymphomagenesis. Blood, 125(8):1272-81
<http://www.bloodjournal.org/content/125/8/1272.long?sso-checked=true>

***

## Identify DEGs: DESeq2

Commonly used tool / package for detection of signficant DEGs include `DESeq2`, `limma/limmavoom`, `edgeR`, etc.

For this workshop, We will demo how to use DESeq2 to detect DEGs between WT and KO groups.

***

### Clean the environment


In [3]:
rm(list=ls())

### Load libraries / packages

In [4]:
##-- List packages required in this analysis
cpan.pkg.list = c('ggplot2', 'scales', 'ape', 'RColorBrewer', 'reshape','VennDiagram')
bioc.pkg.list = c('ctc',  'limma', 'edgeR', 'DESeq2', 'vsn', 'genefilter')

In [5]:
##-- Set up CPAN repo (required if running IRkernel in Jupyter)
cpan.repos = 'http://cran.us.r-project.org'

##-- Install CPAN packages
# install.packages('ggplot2', repos=cpan.repos) ## included in R essentials
# install.packages('scales', repos=cpan.repos) ## included in R essentials
# install.packages('ape', repos=cpan.repos)
# install.packages('RColorBrewer', repos=cpan.repos)
# install.packages('reshape', repos=cpan.repos)
# install.packages('VennDiagram', repos=cpan.repos)

In [6]:
##-- Set up Bioconductor repo
source("http://bioconductor.org/biocLite.R")

##-- Install Bioc packages
# biocLite('ctc')
# biocLite('limma')
# biocLite('edgeR')
# biocLite('DESeq2')
# biocLite('vsn')
# biocLite('genefilter')

Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help


In [7]:
##-- Load libraries
# lapply(c(cpan.pkg.list, bioc.pkg.list),library, character.only = TRUE)
suppressMessages(library('ggplot2'))

### Set up global parameters for the analysis

In [8]:
rep.flag = 1
limma.flag = 1
deseq.flag = 0
deseq2.flag = 1
edger.flag = 1

### Complete workflow
```{r}

## setting
cancer = 'DLBC' 
fdrs = c(0.01, 0.05)
fcs = c(1.5, 2.0) ## 1.4
limmatypes = c('limmavoom', 'limmavoomweighted') ## raw counts (not limma itself!)
colors=c('cold'='cornflowerblue', 'hot'='lightcoral') ## set tumor group colors
sample_ex = c()
# sample_ex = c('S1WT06','S4KO05')
outputpath = 'diff_expr'

outstring = ''
if(length(sample_ex) > 0) {
  outstring = paste0(sample_ex, collapse = '_')
  outstring = paste0('ex',outstring, '.')
}

## paths
path='C:/Users/rbaoc/Desktop/current_proj/CRI_workhop2016/Blood_DLBC/results/degs'
setwd(path)

## files
matrixfile = paste0(cancer, '.raw_counts.txt')
groupfile = paste0(cancer, '.sample_group.txt')
geneinfofile = 'gencode.v24.primary_assembly.annotation.gtf.geneinfo'

indir = '.'
outputpath = paste0(indir, '/', outputpath)

## print info
print(paste0('Cancer = ', cancer))
print(paste0('Marix file = ', matrixfile))
print(paste0('Sample group file  = ', groupfile))
print(paste0('DEG fdr = ', fdrs))
print(paste0('DEG fc = ', fcs))

## ------------------------------------------------------------------------
## import data 
## ------------------------------------------------------------------------

print('Import data ...')

## Read data files 
## note that this expression matrix is already norm + log2 transformed
data = read.table(paste0(indir, '/', matrixfile), sep='\t', header=T, stringsAsFactors=F, na='NA')
group = read.table(paste0(indir, '/', groupfile), sep='\t', header=T, stringsAsFactors=F, na='NA')
geneinfo = read.delim(geneinfofile, header = T, stringsAsFactors = F)

## remove extra columns 
data = data[,c(1,7:12)]
data[1:3,]

colnames(data)[1] = 'Gene'

## add gene symbol (for ensemble geneid)
print(sum(data$Gene %in% geneinfo$gene_id) == nrow(data))

row.names(data) = data[,1]
row.names(group) = group[,1]
row.names(geneinfo) = geneinfo[,1]

data = merge(geneinfo[,c('gene_id', 'gene_name')], data, by = 'row.names')
data[1:4,1:5]

data$Gene = paste0(data$gene_name, '!', data$gene_id)
data = data[,-c(1:3)]
data[1:4,1:5]

## parse data
# colnames(group) = c('Sample', 'Group')
row.names(data) = data[,1]
data = data[,-1]

## remove samples 
data = data[,! colnames(data) %in% sample_ex]
colnames(data)

row.names(group) = group$Sample
group = group[! group$Sample %in% sample_ex,]

## ------------------------------------------------------------------------
## Split by gene type 
## ------------------------------------------------------------------------

## split into coding and noncoding 

head(geneinfo)
data.frame(table(geneinfo$gene_type))

## retrieve gene id only
geneinfo_gene = geneinfo[,c(1,2,4)]
geneinfo_gene = unique(geneinfo_gene)
dim(geneinfo_gene)

geneinfo_gene = data.frame(Key = paste0(geneinfo_gene$gene_name,'!', geneinfo_gene$gene_id), geneinfo_gene)
row.names(geneinfo_gene) = geneinfo_gene[,1]
geneinfo = geneinfo_gene[,-1]

gene_coding = geneinfo_gene[geneinfo_gene$gene_type == 'protein_coding',]
gene_lincrna = geneinfo[geneinfo_gene$gene_type == 'lincRNA',]

dim(gene_coding)
dim(gene_lincrna)

## ------------------------------------------------------------------------
## Select cancer
## ------------------------------------------------------------------------

group_sub = group

type = 'coding'
if(type == 'hg19') {data_sub = data  } else {
  if(type == 'coding') { gene_sub = gene_coding  }
  if(type == 'lincrna') { gene_sub = gene_lincrna }
  
  data_sub = data[row.names(data) %in% row.names(gene_sub),] 
}

dim(data_sub)

## ------------------------------------------------------------------------
## detect DEGs
## ------------------------------------------------------------------------

print('Detect DEGs ...')

## sort matrix sample col to be consistent with group list
data_sub = data_sub[,group_sub$Sample] 
# data_sub_print = data.frame(Gene = row.names(data_sub), data_sub)
# save(data_sub, file=paste0(cancer,'.',matrixfile,'.data.RData'))
# write.table(data_sub_print, file=paste0(cancer,'.',matrixfile,'.data.txt'), row.names = F, col.names = T, quote = F, sep = '\t')

## filter 
dim(data_sub)
# keep = rowSums(data_sub>0) >= ((dim(data_sub)[2]) * 0.5) ## at least 50% samples have >0 counts
# data_sub = data_sub[keep,] 
dim(data_sub)
## 17313   354
# save(data_sub, file=paste0(cancer,'.',matrixfile,'.data.flt.RData'))
# data_sub_print = data.frame(Gene = row.names(data_sub), data_sub)
# write.table(data_sub_print, file=paste0(cancer,'.',matrixfile,'.data.flt.txt'), row.names = F, col.names = T, quote = F, sep = '\t')

## ------------------------------------------------------------------------

if(limma.flag == 1) {
  
  print("Running limma ... ")
  
  ## ------------------------------------------------------------------------
  
  ## filter 
  dim(data_sub)
  keep = rowSums(data_sub>0) >= ((dim(data_sub)[2]) * 0.5) ## at least 50% samples have >0 counts
  data_sub_flt = data_sub[keep,] 
  dim(data_sub_flt)
  ## 17313   354
  # save(data_sub_flt, file=paste0(cancer,'.',matrixfile,'.data.flt.RData'))
  # data_sub_flt_print = data.frame(Gene = row.names(data_sub_flt), data_sub_flt)
  # write.table(data_sub_flt_print, file=paste0(cancer,'.',matrixfile,'.data.flt.txt'), row.names = F, col.names = T, quote = F, sep = '\t')
  
  ## ------------------------------------------------------------------------
  
  ## prepare design matrix and contrasts
  Group = as.factor(group_sub$Group)
  design = model.matrix(~ 0+Group)
  colnames(design) = gsub('Dataset', '', gsub('Group','',colnames(design)))
  head(design)
  
  contrast_matrix = makeContrasts(KO - WT,levels=design)
  
  ## ------------------------------------------------------------------------
  
  for(limmatype in limmatypes) {
    
    print(paste0('limmatype = ', limmatype))
    
    if( limmatype == 'limma' ) { 
      ## create an ExpressionSet obj from a matrix (use Biobase)
      eset = new('ExpressionSet', exprs=as.matrix(data_sub_flt))
      fit = lmFit(eset, design) 
      
      ## ------------------------------------------------------------------------
      
    } else if ( limmatype == 'limmavoom' | limmatype == 'limmavoomweighted') {
      
      dge = DGEList(counts=as.matrix(data_sub_flt))
      #   dim(dge)
      #   keep = rowSums(cpm(dge)>1) >= ((dim(dge)[2]-1) * 0.5) ## at least 50% samples have CPM>=1
      #   dge = dge[keep,] 
      #   dim(dge)
      dge = calcNormFactors(dge, method='TMM') 
      pdf(file=paste0(outputpath, '/', limmatype, '/', cancer,'.',type, '.', outstring,limmatype, '.meanvar.pdf'), width=8, height=6)
      if( limmatype == 'limmavoom') {
        data_voom = voom(dge, design, normalize.method='none', plot=TRUE)
      } else if( limmatype == 'limmavoomweighted') {
        data_voom = voomWithQualityWeights(dge, design, normalize.method='none', plot=TRUE)
      }
      dev.off()
      save(data_voom, file=paste0(outputpath, '/',limmatype, '/',cancer,'.', type, '.',outstring, limmatype,'.RData'))
      data_voom_matrix = data.frame(Gene = row.names(data_voom$E), round(data_voom$E,2)) ## voom norm matrix
      write.table(data_voom_matrix, file=paste0(outputpath, '/',limmatype, '/',cancer,'.', 
                                                type, '.',outstring,
                                                limmatype,'.txt'), 
                  row.names = F, col.names = T, quote = F, sep = '\t')
      
      fit = lmFit(data_voom, design)
    }
    
    ## ------------------------------------------------------------------------
    
    fit2 = contrasts.fit(fit, contrast_matrix)
    fit2 = eBayes(fit2)
    
    ## ------------------------------------------------------------------------
    
    coef_colnames = gsub(' - ', 'vs', colnames(fit2$cov.coefficients))
    deg_lists = list()
    
    for (i in 1:length(coef_colnames)) {
      print(paste0('cov.coefficients col number = ', i))
      comp = coef_colnames[i]
      output = paste0(outputpath,'/',limmatype, '/',cancer, '.', type, '.',
                      outstring,comp, '.', limmatype, '.txt')
      table = topTable(fit2, coef=i, adjust='fdr', number=100000, sort.by='p', resort.by='p', p.value=1, lfc=log2(1.0))
      
      ## ------------------------------------------------------------------------
      
      ## add gene name    
      table$Gene = row.names(table)
      table = table[,c('Gene', colnames(table)[1:6])]
      
      ## ------------------------------------------------------------------------
      
      ## anti-log FC
      table$FC = 0 
      row_pos = which(table$logFC >= 0)
      row_neg = which(table$logFC < 0)
      table$FC[row_pos] = 2^table$logFC[row_pos]
      table$FC[row_neg] = -2^((-1) * table$logFC[row_neg])
      print(sum(table$FC == 0) == 0)
      head(table)
      write.table(table, output, col.names=T, row.names=F, sep='\t', quote=F)
      
      ## ------------------------------------------------------------------------
      
      ## flt DEGs 
      for(fdr in fdrs) {
        for(fc in fcs) {
          print(paste0(fc, ',', fdr))
          table_flt = table[table$adj.P.Val < fdr & (table$logFC >= log2(fc) | 
                                                       table$logFC <= -log2(fc)),]
          
          if(nrow(table_flt) > 0) { 
            write.table(table_flt, paste0(output, '.flt.fdr', fdr, '_fc', fc), 
                        col.names=T, row.names=F, sep='\t', quote=F)
            
            ## print gene symbol only as input for IPA 
            genes = data.frame(Gene = gsub('!\\S+', '', table_flt$Gene), 
                               table_flt[,colnames(table_flt) %in% 
                                           c('logFC', 'P.Value', 'adj.P.Val', 'FC')])
            print(sum(genes$FC == 0) == 0)
            head(genes)
            genes = genes[genes$Gene != '---' & ! is.na(genes$Gene),]
            
            write.table(genes, paste0(output, '.flt.fdr', fdr, '_fc', fc, '.input'), 
                        col.names=T, row.names=F, sep='\t', quote=F)
            write.table(genes[genes$logFC > 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                         '.up.input'), 
                        col.names=T, row.names=F, sep='\t', quote=F)
            write.table(genes[genes$logFC < 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                         '.down.input'), 
                        col.names=T, row.names=F, sep='\t', quote=F)
            
            ## all three statements generate the same output
            # deg_lists = append(deg_lists, list(table_flt[,1]))
            # deg_lists = c(deg_lists, list(table_flt[,1]))
            deg_lists[[i]] = as.vector(table_flt[,1])
            
          }
        }
      }
    }
  }
}

## ------------------------------------------------------------------------

if(deseq.flag == 1)
{
  print("Running DESeq ... ")
  
  caller = 'deseq'
  
  output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                  type, '.',outstring, caller)
  
  ## ------------------------------------------------------------------------

  # detach("package:DESeq2", unload=TRUE)
  library(DESeq) ## conflict with DESeq2
  
  ## ------------------------------------------------------------------------
  
  ## note group_sub must have the same sample order as data_sub
  Group = as.factor(group_sub$Group)
  
  ## Build a count data set
  cds = newCountDataSet( data_sub, Group )
  
  ## ------------------------------------------------------------------------
  
  ## estimate lib size factors
  cds = estimateSizeFactors( cds )
  sizeFactors(cds)
  
  ## print normalized matrix
  cds.before.norm = counts( cds, normalized=F) ## same as data_sub
  cds.after.norm = counts( cds, normalized=T )
  
  cds.before.norm[1:3,1:3]
  cds.after.norm[1:3,1:3]
  
  ## ------------------------------------------------------------------------
  
  ## log2 transformation with Variance stabilizing transformation to estimate n0
  ## NOT for DEG analysis, but for heatmap, clustering etc.
  ## it is important to use 'blind' so the dispersion estimation is not skewed towards design
  
  cdsBlind = estimateDispersions( cds, method="blind" )
  vsd = varianceStabilizingTransformation( cdsBlind )
  vsd.expr = exprs(vsd)
  
  save(vsd, file=paste0(output,'.RData'))
  
  ## save normalized and log2-transformed matrix
  write.table(data.frame(Gene = row.names(vsd.expr), vsd.expr),
              file = paste0(output,'.txt'),
              sep = '\t', col.names = T, row.names = F, quote = F)
  
  ## plot to confirm that vsn removes mean-variance trend 
  # par(mfrow=c(1,2))
  notAllZero = (rowSums(counts(cds))>0)
  
  pdf(paste0(output, '.meanvar.log2.pdf'),width = 7, height = 7)
  meanSdPlot(log2(counts(cds)[notAllZero, ] + 1))
  dev.off()
  
  pdf(paste0(output, '.meanvar.vsd.pdf'),width = 7, height = 7)
  meanSdPlot(vsd[notAllZero, ])
  dev.off()
  
  ## ------------------------------------------------------------------------
  
  ## estimate the dispersions for DEG analysis
  if(rep.flag == 1) { cds = estimateDispersions(cds) }
  if(rep.flag == 0) { cds = estimateDispersions(cds, method="blind", sharingMode="fit-only")}
  
  head( fData(cds) )
  
  save(cds, file=paste0(output,'.dispersion.RData'))
  
  #Plot of estimated dispersions
  pdf(paste0(output, '.dispersion.pdf'),width = 7, height = 7)
  plotDispEsts(cds)
  dev.off()
  
  ## ------------------------------------------------------------------------
  
  group1 = 'KO'
  group2 = 'WT'
  
  comp = paste0(group1, 'vs', group2, '.')
  output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                  type, '.',outstring, comp,caller,'.txt')
  
  ## ------------------------------------------------------------------------
  
  ## filter out low express genes
  rs = rowSums ( counts ( cds ))
  theta = 0.1 ## fraction fo genes to filter out  
  row.select = (rs > quantile(rs, probs=theta))
  table(row.select)
  ## FALSE  TRUE 
  ## 5486 16562 
  
  cds.flt = cds[ row.select, ]
  
  dim(cds)
  dim(cds.flt)
  
  ## ------------------------------------------------------------------------
  
  ## investigate if the threshold (theta) affects DEG result)
  res = nbinomTest(cds, group1, group2)
  
  res.plot = data.frame(
    filterstat = rs,
    pvalue = res$pval,
    row.names = featureNames(cds) )
  head(res.plot)
  
  theta.test = seq(from=0, to=0.5, by=0.1)
  pBH = filtered_p(filter=res.plot$filterstat, 
                   test=res.plot$pvalue, 
                   theta=theta.test, 
                   method="BH")
  head(pBH)
  
  pdf(paste0(output, '.theta.rejection_plot1.pdf'),width = 7, height = 7)
  
  rejection_plot(pBH, at="sample",
                 xlim=c(0, 0.5), ylim=c(0, 2000),
                 xlab="FDR cutoff (Benjamini & Hochberg adjusted p-value)", main="")
  
  dev.off()
  
  pdf(paste0(output, '.theta.rejection_plot2.pdf'),width = 7, height = 7)
  
  rejBH = filtered_R(alpha=0.1, 
                     filter=res.plot$filterstat, 
                     test=res.plot$pvalue, 
                     theta=theta.test, 
                     method="BH")

  plot(theta.test, rejBH, type="l",
       xlab=expression(theta.test), ylab="number of rejections")
  abline(v = theta, col = "red", lwd = 3)
  
  dev.off()
  
  ## ------------------------------------------------------------------------
  
  ## DEG detection (BH for multitesting correction; cannot change ... )
  res = nbinomTest(cds.flt, group1, group2);
  
  save(res, file = paste0(output,'.RData'))
  
  write.table(res,
              file = output,
              sep = '\t', col.names = T, row.names = F, quote = F)
  
  ## ------------------------------------------------------------------------
  
  ## filter DEGs
  
  #remove nan values for the foldchange == NAN
  dim(res)
  res = res[!is.nan(res$foldChange) & ! is.nan(res$padj),];
  dim(res)
  
  for(fdr in fdrs) {
    for(fc in fcs) {
      print(paste0(fc, ',', fdr))
      
      #Plot of MA to display differential expression versus expression strength
      pdf(paste0(output,'.MAplot.pdf'), width=7, height=7)
      #    plotMA(res[!is.infinite(res$log2FoldChange),c(2,6,8)]);
      plotMA(res)
      dev.off()
      
      #Plot of histogram of unadj p-value for sanity check
      pdf(paste0(output,'.pvalue.hist.pdf'), width=7, height=7)
      hist(res$pval, breaks=100, col="skyblue", border="slateblue", main="Histogram of unadjusted p-value", xlab="unajusted p-value") 
      dev.off()   
      
      res.flt = res[(res$foldChange >= fc | res$foldChange <= -fc) & 
                      res$padj < fdr,]
      dim(res.flt)
      
      if(nrow(res.flt) > 0) {
        write.table(res.flt, paste0(output, '.flt.fdr', fdr, '_fc', fc), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        
        ## print gene symbol only as input for IPA 
        genes = data.frame(id = gsub('!\\S+', '', res.flt$id), 
                           res.flt[,colnames(res.flt) %in% 
                                       c('log2FoldChange', 'pval', 'padj', 'foldChange')])
        print(sum(genes$foldChange == 0) == 0)
        head(genes)
        genes = genes[genes$id != '---' & ! is.na(genes$id),]
        
        write.table(genes, paste0(output, '.flt.fdr', fdr, '_fc', fc, '.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        write.table(genes[genes$log2FoldChange > 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                     '.up.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        write.table(genes[genes$log2FoldChange < 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                     '.down.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
      }
    }
  }
}

## ------------------------------------------------------------------------

if(deseq2.flag == 1)
{
  
  print("Running DESeq2 ... ")
  
  caller = 'deseq2'
  
  output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                  type, '.',outstring, caller)
  
  ## ------------------------------------------------------------------------
  
  if(deseq.flag == 1) { detach("package:DESeq", unload=TRUE) }
  library(DESeq2) ## conflict with DESeq

  ## ------------------------------------------------------------------------
  
  ## note group_sub must have the same sample order as data_sub
  Group = as.factor(group_sub$Group)
  
  ## Build a count matrix
  cds = as.matrix(data_sub)
  
  dds = DESeqDataSetFromMatrix(countData = cds,
                               colData = group_sub, 
                               design = ~ Group)
  
  # ## note this is just simple filter to reduce mem; no affect on DEG results
  # dim(dds)
  # dds = dds[ rowSums(counts(dds)) > 0, ]
  # dim(dds)
  
  # #Plot of estimated dispersions
  # pdf(paste0(output, '.dispersion.pdf'),width = 7, height = 7)
  # plotDispEsts(dds)
  # dev.off()
  
  ## ------------------------------------------------------------------------

  ## norm matrix for heatmap clusterin etc. three methods ...

  rld = rlog(dds, blind=FALSE)
  vsd = varianceStabilizingTransformation(dds, blind=FALSE)
  # vsd.fast = vst(dds, blind=FALSE)
  
  head(assay(rld), 3)
  
  write.table(data.frame(Gene = row.names(assay(rld)), assay(rld)),
              file = paste0(output, '.rld.txt'),
              sep = '\t', col.names = T, row.names = F, quote = F)
  
  write.table(data.frame(Gene = row.names(assay(vsd)), assay(vsd)),
              file = paste0(output, '.vsd.txt'),
              sep = '\t', col.names = T, row.names = F, quote = F)
  
  ## ------------------------------------------------------------------------
  
  ## plot mean to var to confirm variance shrink works 
  notAllZero = (rowSums(counts(dds))>0)
  
  pdf(paste0(output, '.meanvar.log2.pdf'),width = 7, height = 7)
  meanSdPlot(log2(counts(estimateSizeFactors(dds),normalized=TRUE)[notAllZero,] + 1))
  dev.off()
  
  pdf(paste0(output, '.meanvar.rld.pdf'),width = 7, height = 7)
  meanSdPlot(assay(rld[notAllZero,]))
  dev.off()
  
  pdf(paste0(output, '.meanvar.vsd.pdf'),width = 7, height = 7)
  meanSdPlot(assay(vsd[notAllZero,]))
  dev.off()
  
  ## ------------------------------------------------------------------------
  
  ## plot sample correlation 
  
  sampleDists = dist(t(assay(rld)))
  
  sampleDistMatrix = as.matrix(sampleDists)
  rownames(sampleDistMatrix) = paste(rld$Group, rld$Libtype, sep="-")
  colnames(sampleDistMatrix) = rownames(sampleDistMatrix)
  
  colors = rev(cm.colors(32))[1:16]
  
  pdf(paste0(output, '.sm_cor.pdf'),width = 7, height = 7)
  pheatmap(sampleDistMatrix,
           clustering_distance_rows=sampleDists,
           clustering_distance_cols=sampleDists,
           col=colors)
  dev.off()
  
  ## ------------------------------------------------------------------------
  
  group1 = 'KO'
  group2 = 'WT'
  
  comp = paste0(group1, 'vs', group2, '.')
  output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                  type, '.',outstring, comp,caller,'.txt')
  
  ## DEG analysis
  dds = DESeq(dds, test="Wald", betaPrior=T)
  res = results(dds, contrast=c("Group",group1,group2), pAdjustMethod ="fdr", alpha=fdr)
  
  save(res, file = paste0(output,'.RData'))
  
  ## ------------------------------------------------------------------------
  
  ## Plot of MA to display differential expression versus expression strength
  pdf(paste0(output,'.MAplot.pdf'), width=7, height=7)
  plotMA(res, main="DESeq2", ylim=c(-2,2))
  dev.off()
  
  ## print more info of the test
  mcols(res)$description

  ## ------------------------------------------------------------------------
  
  head(res)
  summary(res)
  
  res = as.data.frame(res)
  
  ## ------------------------------------------------------------------------
  
  ## anti-log FC
  res$foldChange = NA
  row_pos = which(! is.na(res$log2FoldChange) & res$log2FoldChange >= 0)
  row_neg = which(! is.na(res$log2FoldChange) & res$log2FoldChange < 0)
  res$foldChange[row_pos] = 2^res$log2FoldChange[row_pos]
  res$foldChange[row_neg] = -2^((-1) * res$log2FoldChange[row_neg])
  print(sum(res$foldChange == 0) == 0)
  head(res)
  
  res = data.frame(id = row.names(res), res)

  write.table(res,
              file = output,
              sep = '\t', col.names = T, row.names = F, quote = F)
  
  ## ------------------------------------------------------------------------
  
  ## filter DEGs
  
  #remove nan values for the foldchange == NAN
  dim(res)
  res = res[!is.na(res$foldChange) & ! is.na(res$padj),];
  dim(res)
  
  for(fdr in fdrs) {
    for(fc in fcs) {
      print(paste0(fc, ',', fdr))
      
      #Plot of histogram of unadj p-value for sanity check
      pdf(paste0(output,'.pvalue.hist.pdf'), width=7, height=7)
      hist(res$pval, breaks=100, col="skyblue", border="slateblue", main="Histogram of unadjusted p-value", xlab="unajusted p-value") 
      dev.off()   
      
      res.flt = res[(res$foldChange >= fc | res$foldChange <= -fc) & 
                      res$padj < fdr,]
      dim(res.flt)
      
      if(nrow(res.flt) > 0) {
        write.table(res.flt, paste0(output, '.flt.fdr', fdr, '_fc', fc), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        
        ## print gene symbol only as input for IPA 
        genes = data.frame(id = gsub('!\\S+', '', res.flt$id), 
                           res.flt[,colnames(res.flt) %in% 
                                     c('log2FoldChange', 'pval', 'padj', 'foldChange')])
        print(sum(genes$foldChange == 0) == 0)
        head(genes)
        genes = genes[genes$id != '---' & ! is.na(genes$id),]
        
        write.table(genes, paste0(output, '.flt.fdr', fdr, '_fc', fc, '.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        write.table(genes[genes$log2FoldChange > 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                              '.up.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
        write.table(genes[genes$log2FoldChange < 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                              '.down.input'), 
                    col.names=T, row.names=F, sep='\t', quote=F)
      }
    }
  }
}

## ------------------------------------------------------------------------

if(edger.flag == 1)
{
  print("Running egdeR ... ")
  
  caller = 'edger'
  
  output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                  type, '.',outstring, caller)
  
  if (rep.flag == 1)
  {
 
    Group = as.factor(group_sub$Group)
    
    counts = as.matrix(data_sub)
    
    ## ------------------------------------------------------------------------
    
    #computes counts permilion with cpm function 
    cpms = cpm(counts, log=F) 
    
    #In edgeR, it is recommended to remove features without at least 1 read per million in n of the samples, where n is the size of the smallest group of replicates
    
    keep = rowSums(cpms >1) >= min(data.frame(table(Group))[,2])
    
    dim(counts)
    counts = counts[keep,]
    dim(counts)
    
    ## ------------------------------------------------------------------------
    
    #create a DGEList object
    d = DGEList(counts=counts, group=Group) 
    
    #estimate normalization factors
    d = calcNormFactors(d, method='TMM');
    
    #estimate common negative binomial dispersion by conditional maximum likelihood
    ## might need to add one more flag for this argument in estimateCommonDisp rowsum.filter= value for the filtering out low abundance tags in the est of the common dispersion
    d = estimateCommonDisp(d, tol=1e-6);
    #estimate empirical bayes tagwise dispersion values
    d = estimateTagwiseDisp(d, trend="loess", method="grid")
    
    ## ------------------------------------------------------------------------
    
    ## print norm matrix
    logcpm = cpm(d, prior.count=2, log=TRUE)
    
    save(logcpm, file = paste0(output, '.RData'))
    
    write.table(data.frame(Gene = row.names(logcpm), logcpm),
                file = paste0(output, '.txt'),
                sep = '\t', col.names = T, row.names = F, quote = F)
    
    ## ------------------------------------------------------------------------
    
    #mean-variance plot. visual representation of the mean-variance relationship with plotMeanVar
    pdf(paste0(output,'.meanvar.pdf'), width=7, height=7)
    plotMeanVar(d, show.tagwise.vars=T, NBline=TRUE);
    dev.off()
    
    #mean-variance plot. visual representation of the mean-variance relationship with plotBCV
    pdf(paste0(output,'.bcv.pdf'), width=7, height=7)
    plotBCV(d);
    dev.off()  
    
    ## ------------------------------------------------------------------------
    
    group1 = 'KO'
    group2 = 'WT'
    
    comp = paste0(group1, 'vs', group2, '.')
    output = paste0(outputpath, '/', caller,'/',cancer,'.', 
                    type, '.',outstring, comp,caller,'.txt')
    
    de = exactTest(d, pair=c(group1, group2));
    
    #adjust.method = "BH","BY","fdr". FWER method = "holm", "hochberg", "hommel", "bonferroni"
    tt = topTags(de, n=nrow(d), adjust.method ="fdr")  
    
    rn = rownames(tt$table);
    
    res = tt$table
    
    ## ------------------------------------------------------------------------
    
    head(res)
    summary(res)
    
    res = as.data.frame(res)
    
    ## ------------------------------------------------------------------------
    
    ## anti-log FC
    res$FC = NA
    row_pos = which(! is.na(res$logFC) & res$logFC >= 0)
    row_neg = which(! is.na(res$logFC) & res$logFC < 0)
    res$FC[row_pos] = 2^res$logFC[row_pos]
    res$FC[row_neg] = -2^((-1) * res$logFC[row_neg])
    print(sum(res$FC == 0) == 0)
    head(res)
    
    res = data.frame(Gene = row.names(res), res)
    
    write.table(res,
                file = output,
                sep = '\t', col.names = T, row.names = F, quote = F)
    
    ## ------------------------------------------------------------------------
    
    ## filter DEGs
    
    #remove nan values for the foldchange == NAN
    dim(res)
    res = res[!is.na(res$FC) & ! is.na(res$FDR),];
    dim(res)
    
    for(fdr in fdrs) {
      for(fc in fcs) {
        print(paste0(fc, ',', fdr))
        
        #Plot of histogram of unadj p-value for sanity check
        pdf(paste0(output,'.pvalue.hist.pdf'), width=7, height=7)
        hist(res$PValue, breaks=100, col="skyblue", border="slateblue", main="Histogram of unadjusted p-value", xlab="unajusted p-value") 
        dev.off()   
        
        #creat a graphical summary for DEG
        pdf(file = paste0(output,'.flt.fdr',fdr,'.pdf') ,width=7,height=7)    
        deg = rn[tt$table$FDR < fdr];
        plotSmear(d, de.tags=deg)
        dev.off()
        
        res.flt = res[(res$FC >= fc | res$FC <= -fc) & 
                        res$FDR < fdr,]
        dim(res.flt)
        
        if(nrow(res.flt) > 0) {
          write.table(res.flt, paste0(output, '.flt.fdr', fdr, '_fc', fc), 
                      col.names=T, row.names=F, sep='\t', quote=F)
          
          ## print gene symbol only as input for IPA 
          genes = data.frame(id = gsub('!\\S+', '', res.flt$Gene), 
                             res.flt[,colnames(res.flt) %in% 
                                       c('logFC', 'PValue', 'FDR', 'FC')])
          print(sum(genes$FC == 0) == 0)
          head(genes)
          genes = genes[genes$id != '---' & ! is.na(genes$id),]
          
          write.table(genes, paste0(output, '.flt.fdr', fdr, '_fc', fc, '.input'), 
                      col.names=T, row.names=F, sep='\t', quote=F)
          write.table(genes[genes$logFC > 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                                '.up.input'), 
                      col.names=T, row.names=F, sep='\t', quote=F)
          write.table(genes[genes$logFC < 0, ], paste0(output, '.flt.fdr', fdr, '_fc', fc, 
                                                                '.down.input'), 
                      col.names=T, row.names=F, sep='\t', quote=F)
        }
      }
    }
    
  }
}

#########################################################

print('...Done!')

sessionInfo()

```